## Spatial Interpolation

In this Notebook, data obtained from the Atmospheric Radiation Measurement (ARM) repository collected by the G-1 aircraft of the GOAmazon 2014/15 project are interpolated and generate a dataset for each variable.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from statistics import mean

In [2]:
def read_files(season):
    """
    Read the files and filter by flight number
    """
    if season == 'dry':
        g1_d17 = pd.read_excel("G1_dryseason_17.xlsx")
        g1_d18 = pd.read_excel("G1_dryseason_18.xlsx")
        g1_d19 = pd.read_excel("G1_dryseason_19.xlsx")
        g1_d20 = pd.read_excel("G1_dryseason_20.xlsx")
        g1_d21 = pd.read_excel("G1_dryseason_21.xlsx")
        g1_d22 = pd.read_excel("G1_dryseason_22.xlsx")
        g1_d23 = pd.read_excel("G1_dryseason_23.xlsx")
        g1_d24 = pd.read_excel("G1_dryseason_24.xlsx")
        g1_d25 = pd.read_excel("G1_dryseason_25.xlsx")
        g1_d26 = pd.read_excel("G1_dryseason_26.xlsx")
        g1_d27 = pd.read_excel("G1_dryseason_27.xlsx")
        g1_d28 = pd.read_excel("G1_dryseason_28.xlsx")
        g1_d29 = pd.read_excel("G1_dryseason_29.xlsx")
        g1_d30 = pd.read_excel("G1_dryseason_30.xlsx")
        g1_d31 = pd.read_excel("G1_dryseason_31.xlsx")
        g1_d32 = pd.read_excel("G1_dryseason_32.xlsx")
        g1_d33 = pd.read_excel("G1_dryseason_33.xlsx")
        
        flights = [g1_d17,g1_d18,g1_d19,g1_d20,g1_d21,g1_d22,g1_d23,g1_d24,g1_d25,g1_d26,g1_d27,g1_d28,g1_d29,g1_d30,g1_d31,g1_d32,g1_d33]
        return flights
    
    elif season == 'wet':
        g1_d1 = pd.read_excel("G1_wetseason_1.xlsx")
        g1_d2 = pd.read_excel("G1_wetseason_2.xlsx")
        g1_d3 = pd.read_excel("G1_wetseason_3.xlsx")
        g1_d4 = pd.read_excel("G1_wetseason_4.xlsx")
        g1_d5 = pd.read_excel("G1_wetseason_5.xlsx")
        g1_d6 = pd.read_excel("G1_wetseason_6.xlsx")
        g1_d7 = pd.read_excel("G1_wetseason_7.xlsx")
        g1_d8 = pd.read_excel("G1_wetseason_8.xlsx")
        g1_d9 = pd.read_excel("G1_wetseason_9.xlsx")
        g1_d10 = pd.read_excel("G1_wetseason_10.xlsx")
        g1_d11 = pd.read_excel("G1_wetseason_11.xlsx")
        g1_d12 = pd.read_excel("G1_wetseason_12.xlsx")
        g1_d13 = pd.read_excel("G1_wetseason_13.xlsx")
        g1_d14 = pd.read_excel("G1_wetseason_14.xlsx")
        g1_d15 = pd.read_excel("G1_wetseason_15.xlsx")
        g1_d16 = pd.read_excel("G1_wetseason_16.xlsx")
        
        flights = [g1_d1,g1_d2,g1_d3,g1_d4,g1_d5,g1_d6,g1_d7,g1_d8,g1_d9,g1_d10,g1_d11,g1_d12,g1_d13,g1_d14,g1_d15,g1_d16]
        return flights

    
def select_colums(df_flight, variable):
    """
    Select only the relevant columns of the dataframe
    """
    
    df_flight = df_flight.drop(columns=['Unnamed: 0'])
    df_flight = df_flight[['lat','long','alt',variable]]
    df_flight = df_flight.dropna(subset=[variable])
    
    return df_flight


def filter_alt(df_flight):
    """
    Filter the dataframe by altitude
    """
    
    max_alt = 1500;
    df_flight = df_flight[df_flight['alt']<=max_alt].reset_index(drop=True)
    
    return df_flight


def maxmin_season(season, flights):
    """
    Calculate the maximum and minimum from latitude and logitude of each season 
    """

    l_max_lat = []
    l_max_lon = []
    l_min_lat = []
    l_min_lon = []

    for flight in flights:
        l_max_lat.append(flight['lat'].max())
        l_min_lat.append(flight['lat'].min())
        l_max_lon.append(flight['long'].max())
        l_min_lon.append(flight['long'].min())
    max_lat = max(l_max_lat)
    max_lon = max(l_max_lon)
    min_lat = min(l_min_lat)
    min_lon = min(l_min_lon)

    return max_lat, max_lon, min_lat, min_lon


def dxdy(df_flight):
    """
    Calculate the discretization of latitude and longitude
    """
    
    dxs = []
    dys = []

    for i in range(len(df_flight)-1):
        dlat = abs(df_flight['lat'][i]-df_flight['lat'][i+1])
        dys.append(dlat)
        dlong = abs(df_flight['long'][i]-df_flight['long'][i+1])
        dxs.append(dlong)
    
    return max(dys), max(dxs)


def dxdy_season(season, flights):
    """
    Calculate the discretization of latitude and longitude for the season
    """   
    
    dxs = []
    dys = []

    for flight in flights:
        dxs.append(dxdy(flight)[0])
        dys.append(dxdy(flight)[1])
    dx = min(dxs)
    dy = min(dys)
        
    return dx,dy


def interpolate(df_flight,variable,max_lat,max_lon,min_lat,min_lon,dx,dy):
    """
    Interpolate the variable under the specified conditions
    """
    
    # Coordinates and variable
    x = df_flight['long']
    y = df_flight['lat']
    z = df_flight[variable]

    # Create the mesh
    xi = np.arange(min_lon,max_lon,dx) 
    yi = np.arange(min_lat,max_lat,dy)
    xi,yi = np.meshgrid(xi,yi)
    xi_ = np.arange(min_lon,max_lon,dx) 
    yi_ = np.arange(min_lat,max_lat,dy)

    # Interpolate
    zi = griddata((x,y),z,(xi,yi),method='linear')
    
    return zi,xi_,yi_,x,y,dx,dy


def plot_interpolation(variable,xi,yi,zi,x,y):
    """
    Plot the result of the interpolation
    """
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    plt.contourf(xi,yi,zi)
    plt.plot(x,y,'k.')
    plt.xlabel('Longitude (°)')
    plt.ylabel('Latitude (°)')
    plt.title('Intepolation of {}'.format(variable))
    plt.colorbar()
    

def create_dataframe(min_lon,max_lon,dx,min_lat,max_lat,dy,zi):
    """
    Create the dataframe resulting of the interpolation 
    """
    
    xi = np.arange(min_lon,max_lon,dx) 
    yi = np.arange(min_lat,max_lat,dy)
    
    matriz = []
    for i in range(len(yi)):
        linha = []
        for j in range(len(xi)):
            linha.append(zi[i][j])
        matriz.append(linha) 
    interp = pd.DataFrame(matriz)
    interp.index = yi
    interp.columns = xi
    
    return interp


def interpolate_variable(df_flight, variable, season, flights):
    """
    Interpolate the variable for the selected season
    """
    
    # Select columns
    df_flight = select_colums(df_flight, variable)
    
    if not df_flight.empty:
        # Filter the altitude
        f_df_flight = filter_alt(df_flight)
        
        if f_df_flight.shape[0] > 100:

            # Calculate the maximum and minimum of latitude and longitude
            max_lat, max_lon, min_lat, min_lon = maxmin_season(season, flights)

            # Calculate the discretization
            dx,dy = dxdy_season(season, flights)

            # Interpolate
            zi,xi,yi,x,y,dx,dy = interpolate(f_df_flight,variable,max_lat,max_lon,min_lat,min_lon,dx,dy)

            # Plot the interpolation
            #plot_interpolation(variable,xi,yi,zi,x,y)

            # Create dataframe
            interp = create_dataframe(min_lon,max_lon,dx,min_lat,max_lat,dy,zi)

            return interp
        
    
def summarization(min_lon,max_lon,dx,min_lat,max_lat,dy,list_flights):
    """
    Summarize the interpolations of the variable in the season
    """
    
    xi = np.arange(min_lon,max_lon,dx) 
    yi = np.arange(min_lat,max_lat,dy)
    
    z_final = []
    
    # Threshold 
    minimum = len(list_flights)/2;
    
    for l in range(len(list_flights)):
        if type(list_flights[l]) == pd.core.frame.DataFrame:
            flight_df = list_flights[l]
            
    for i in range(len(flight_df)):
        line = []
        for j in range(len(flight_df.columns)):
            counter = 0
            lista = []
            for k in range(len(list_flights)):
                flight = list_flights[k]
                if type(flight) == pd.core.frame.DataFrame:
                    if np.isnan(flight.iloc[i,j]) == False:
                        lista.append(flight.iloc[i,j])
                        counter += 1
            if len(lista) > minimum:
                idmax = lista.index(max(lista))
                lista[idmax] = 0
                line.append(mean(lista))
            else:
                line.append(None)
        z_final.append(line)
    
    z_final = pd.DataFrame(z_final)
    z_final.index = yi
    z_final.columns = xi

    return xi,yi,z_final


def plot_summarization(variable,xi,yi,z_final):
    """
    Plot the summarization 
    """
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    plt.contourf(xi,yi,z_final)
    plt.xlabel('Longitude (°)')
    plt.ylabel('Latitude (°)')
    plt.title('Interpolation of {}'.format(variable))
    plt.colorbar()
    

def interpolation_season(variable, season):
    """
    Interpolate the variable during the season
    """
    
    # Read the files
    if season == 'dry':
        flights = read_files(season) 
    elif season == 'wet':
        flights = read_files(season)
    
    # Calculate the maximum and minimum of latitude and longitude
    max_lat, max_lon, min_lat, min_lon = maxmin_season(season, flights)

    # Calculate the discretization 
    dx,dy = dxdy_season(season, flights)
    
    lista_flights = []
    for flight in flights:
        interpolation = interpolate_variable(flight, variable, season, flights)
        lista_flights.append(interpolation)

    # Summarize the interpolations
    xi,yi,z_final = summarization(min_lon,max_lon,dx,min_lat,max_lat,dy,lista_flights)

    # Plot 2D
    #plot_summarization(variable,xi,yi,z_final)
    
    return z_final


def generate_dataset(season, list_variables, file_name):
    """
    Generates the interpolation for each variable from the list and a dataframe
    """
    
    # Iterates and generates the interpolation for each variable
    for i in range(len(list_variables)):
        if i == 0:
            df_season = interpolation_season(list_variables[i], season)
        
            # Stack the dataset
            df_season = pd.DataFrame(df_season.stack(dropna=False)).reset_index()
            df_season = df_season.rename(columns={'level_0': 'latitude', 'level_1': 'longitude', 0: list_variables[i]})
        
        else:
            df_interpolated = interpolation_season(list_variables[i], season)
        
            # Stack the dataset
            df_interpolated = pd.DataFrame(df_interpolated.stack(dropna=False)).reset_index()
            df_interpolated = df_interpolated.rename(columns={'level_0': 'latitude', 'level_1': 'longitude', 0: list_variables[i]})
        
            # Merge
            df_season = df_season.merge(df_interpolated, on=['latitude', 'longitude'])
            
    # Filter for not NaN
    df_season = df_season.dropna(subset=list_variables, how='all').reset_index(drop=True)
    
    # Export data
    df_season.to_excel('{}.xlsx'.format(file_name), index=False)
    
    return df_season        

In [3]:
list_variables = ['co(ppm)', 'o3(ppb)', 'nox(ppb)', 'CPC3010','Acetonitrile(ppb)', 'Isoprene(ppb)']
season = 'dry'
file_name = 'Interpolated_Data'
dataset = generate_dataset(season, list_variables, file_name)
dataset

,latitude,longitude,co(ppm),o3(ppb),nox(ppb),CPC3010,Acetonitrile(ppb),Isoprene(ppb)
0,-3.6317,-60.411,0.130965,NaN,1.559209,NaN,NaN,NaN
1,-3.6317,-60.410,0.130779,NaN,1.557812,NaN,NaN,NaN
2,-3.6317,-60.409,0.130844,NaN,1.546705,NaN,NaN,NaN
3,-3.6297,-60.413,0.132191,NaN,1.688251,NaN,NaN,NaN
4,-3.6297,-60.412,0.132132,NaN,1.678755,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
207885,-2.8597,-60.185,0.129748,NaN,NaN,NaN,NaN,NaN
207886,-2.8597,-60.184,0.12924,NaN,NaN,NaN,NaN,NaN
207887,-2.8597,-60.183,0.128472,NaN,NaN,NaN,NaN,NaN
207888,-2.8597,-60.182,0.128627,NaN,NaN,NaN,NaN,NaN
